In [23]:
import pandas as pd
import openpyxl
import openpyxl.drawing.image
import json

def extract_images_from_xlsx(file_path):
    workbook = openpyxl.load_workbook(filename=file_path)
    images = []

    for sheet in workbook.worksheets:
        for drawing in sheet._images:
            if isinstance(drawing, openpyxl.drawing.image.Image):
                images.append(drawing)

    return images

def read_xls_with_images(file_path):
    df = pd.read_excel(file_path, keep_default_na=False)
    images = extract_images_from_xlsx(file_path)
    assert len(df) == len(images)
    df["image"] = images
    return df

def process(x):
    response = json.loads(x)
    s = response["choices"][0]["message"]["content"].lstrip("```json\n").rstrip("\n```")
    item = json.loads(s)
    exist = item["exist"]
    primary = secondary = freshness = ""
    if exist == "是":
        primary = item["category"]["primary"]
        secondary = item["category"]["secondary"]
        freshness = item["freshness"]
    return exist, primary, secondary, freshness, response["usage"]["total_tokens"], json.dumps(item, ensure_ascii=False), x


def main():
    ifile = "data.xlsx"
    ofile = "data2.xlsx"
    df = read_xls_with_images(ifile)
    
    df = df.head(91)
    results = pd.DataFrame(open("output.jsonl").readlines())
    df["results"] = results.loc[:,0]
    
    df["exist"], df["primary"], df["secondary"], df["freshness"], df["tokens"], df["json"], df["model_output"] = zip(*df["results"].map(process))

    df.drop('image', axis=1).drop('results', axis=1).to_excel(ofile, index=False)
    
    workbook = openpyxl.load_workbook(ofile)
    sheet = workbook.active

    sheet.column_dimensions['E'].width = 20

    for i in range(2, len(df) + 2):
        sheet.row_dimensions[i].height = 160

    for index, row in df.iterrows():
        image = row['image']
        image.width = 160
        image.height = 200
        cell = 'E' + str(index + 2)
        sheet.add_image(image, cell)

    workbook.save(ofile)


In [24]:
main()